In [2]:
from helpers import *
from rascal.representations import SphericalCovariants 
from rascal.neighbourlist.structure_manager import mask_center_atoms_by_species #change later
from rascal.representations import SphericalExpansion
from rascal.utils import ClebschGordanReal, compute_lambda_soap, spherical_expansion_reshape, xyz_to_spherical
from rascal.representations import SphericalInvariants
from sklearn.metrics import mean_squared_error, mean_absolute_error



In [3]:
hypers = {"interaction_cutoff": 3,
          "radial_basis": "GTO",
          "max_radial": 9,
          "max_angular": 9,
          "gaussian_sigma_constant": 0.3,
          "gaussian_sigma_type":"Constant",
          "cutoff_function_type":"ShiftedCosine",
          "cutoff_smooth_width": 0.5,
          "compute_gradients":False,
          "cutoff_function_parameters":dict(rate=1,scale=3.5,exponent=4),
         }
hypers_soap = {"soap_type": "PowerSpectrum",
          "interaction_cutoff": 3,
          "radial_basis": "GTO",
          "max_radial": 9,
          "max_angular": 9,
          "gaussian_sigma_constant": 0.3,
          "gaussian_sigma_type":"Constant",
          "cutoff_function_type":"ShiftedCosine",
          "cutoff_smooth_width": 0.5,
          "normalize": True,
          "compute_gradients":False,
          "cutoff_function_parameters":dict(rate=1,scale=3.5,exponent=4),
         }

In [4]:
def build_lambda(structures,hypers):
    spex = SphericalExpansion(**hypers)
    feat_scaling = 1e6            # just a scaling to make coefficients O(1)
    feats = spex.transform(structures).get_features(spex)
    ref_feats = feat_scaling*spherical_expansion_reshape(feats, **hypers)
    CG = ClebschGordanReal(lmax=hypers["max_angular"])
    sel_lambda = 2
    lsoap_utils = compute_lambda_soap(ref_feats, CG, sel_lambda, 1)
    return lsoap_utils.reshape(lsoap_utils.shape[0],5,-1)

def f2v(vec):
    t = ()
    for s in vec.shape:
        t += (s//5,5)
    return vec.reshape(t)

def v2f(vec):
    t = ()
    for s in vec.shape[:-1:2]:        
        t += tuple([s*5])
    if len(vec.shape)%2==1: #if the number of dimensions is uneven
        t += tuple([-1])
    #print(t)
    return vec.reshape(t)

def train_model(X_train, y_train, sigma=1e-03):
    KMM = f2v(v2f(X_train) @ v2f(X_train).T)
    A = v2f(KMM)
    weights = f2v( np.linalg.lstsq(A + sigma * np.eye(A.shape[0]), v2f(y_train), rcond=None)[0] )
    return weights

def train_soap(X_train,y_train):
    kernel = (X_train@X_train.T)**1
    weights = np.linalg.lstsq( kernel + 1e-3*np.eye(kernel.shape[0]), y_train, rcond=None)[0]
    return weights

def predict_soap(X_test,X_train,weights):
    kernel = (X_test@X_train.T)**1
    return kernel @ weights

def predict(X_test, X_train, weights):
    lKM = f2v(v2f(X_test) @ v2f(X_train).T)
    y_pred = f2v(v2f(lKM) @ v2f(weights) )
    return y_pred

In [13]:
A = np.eye(2*5*2*5)

In [15]:
A

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [5]:
PATH_TRAIN = "./train_tensor/CSD-3k+S546_shift_tensors.xyz"
PATH_TEST = "./test_tensor/CSD-500+104-7_shift_tensors.xyz"

structures_train, tensors_train = load_CSD_data(PATH_TRAIN,prop_string="cs_tensor",random_subsample=20)
structures_test, tensors_test = load_CSD_data(PATH_TEST,prop_string="cs_tensor",random_subsample=20)#,random_subsample=20)


#-----mask----
for structure in structures_train: mask_center_atoms_by_species(structure,species_select=[1])
for structure in structures_test: mask_center_atoms_by_species(structure,species_select=[1])

#-----select tensors-----
train_tensors = np.array([tensor for structure in structures_train for tensor in structure.arrays["cs_tensor"][structure.arrays["center_atoms_mask"]]])
test_tensors = np.array([tensor for structure in structures_test for tensor in structure.arrays["cs_tensor"][structure.arrays["center_atoms_mask"]]])

#------transform tensors
cg = ClebschGordanReal(lmax=2)
tensors_train_coupled = cg.couple(xyz_to_spherical(train_tensors.reshape(train_tensors.shape[0],3,3)))
train_tensors_trans = tensors_train_coupled[(1,1)][2]

tensors_test_coupled = cg.couple(xyz_to_spherical(test_tensors.reshape(test_tensors.shape[0],3,3)))
test_tensors_trans = tensors_test_coupled[(1,1)][2]

###-----lambda=2-------
X_train = build_lambda(structures_train,hypers)
X_test = build_lambda(structures_test,hypers)

#-----train model for lambda=2------
weights = train_model(X_train, train_tensors_trans)
y_pred = predict(X_test,X_train,weights)


In [6]:
test_tensors_trans

array([[-1.12521902, -0.55260395,  0.25397126, -1.98534371,  0.38091842],
       [ 1.12811816,  0.54871486,  0.2556859 , -1.98675792,  0.38176695],
       [-1.12521902, -0.55260395,  0.25397126, -1.98534371,  0.38091842],
       ...,
       [-3.62526576,  4.96834438,  4.65219339, -3.33761472, -1.20936473],
       [-0.53938105, -0.04970961,  2.98274366,  4.51699812,  4.27884455],
       [ 0.53938105,  0.04970961,  2.98274366,  4.51699812,  4.27884455]])

In [7]:
y_pred

array([[-292.41765499,  192.70286943,    0.        , -119.63180183,
           0.        ],
       [-342.22134746,  171.88116508,    0.        ,  -41.30900212,
           0.        ],
       [-292.40830452,  192.67330828,    0.        , -119.64078283,
           0.        ],
       ...,
       [   6.16603156, -315.95919229,    0.        ,    0.        ,
           0.        ],
       [ 194.61824219, -742.11742675,    0.        ,  -37.37815157,
           0.        ],
       [ 234.42091028, -606.2672545 ,    0.        ,  -48.33771909,
           0.        ]])

In [8]:
y_pred[0]-test_tensors_trans[0]

array([-2.91292436e+02,  1.93255473e+02, -2.53971261e-01, -1.17646458e+02,
       -3.80918423e-01])

In [9]:
def rmse_tens(y_true,y_pred):
    return np.sqrt(np.average([np.linalg.norm(i[0]-i[1])**2 for i in zip(y_true,y_pred)]))

In [10]:
rmse_tens(y_pred,test_tensors_trans)

515.4703021574411

In [11]:
A = np.arange(3*3*5)
A = A.reshape(5,3,3)

np.linalg.norm(A,axis=2).shape

(5, 3)

In [12]:
A.shape

(5, 3, 3)

In [ ]:
f
